### Author: Md Fahim Hasan
### Work Email: mdfahim.hasan@bayer.com

# Read me

The weather `4km Analytical Ready Data (ARD)`s consists of the folowing datasets-
  1. min temperature
  2. max temperature
  3. average temperature
  4. average wind speed
  5. average relative humidity
  6. total precipitation (modeled or resampled) **
  7. min relative humidity
  8. max relative humidity
  9. total net solar radiation

From these variables, `1-6` will be interpolated/downscaled by ML/DL models from ERA5 28km spatial resolution to 4km resolution. Due to lack of observed (training) TWC data, variables `7-9` couldn't be interpolated by model, therefore, we will resample the data (bilinear resampling) to 4km spatial resolution. All these datasets will be converted to a dataframe/parquet/csv format with date, latitude, and longitude records to generate the ARDs.

### __**Note:__
- We have TWC high resolution data records starting from 2015-06-30 (for precipitation from 2015-12-01). We have modeled high resolution data from 2002 to 2023. Despite that, from 2015 we will use TWC high resolution data. Therefore, we will replace modeled data with TWC datasets from 2015 before compiling all datasets into the 4km ARD.
- The precipitation model didn't perform well. So, we will see how the environmental cluster model performs with modeled precipitation `(V1)` and resampled (directly from ERA5) precipitation `(V2)`.
- The 4km ARD versions will have data compiled (averaged for each date (month_day)) from 2002-2023. 
- The 4km ARD is processed for the whole ROI (part of Central valley, California).
- The functions used in this script can be found in `ARD_utils.ipynb` script.
- Used a model generated 4km resolution dataset (max_temperature) as the reference raster for resampling TWC High-Res precipitation data from 8km to 4km.

## 4km ARD Processing Steps:

The following steps are followed to compile 4km weather datasets into the ARD-
1. __Copying 7-9 datasets to '../datasets/downscaled_data' directory:__ The `min relative humidity`, `max relative humidity`, and `total net solar radiation` datasets were not modeled. These datasets were resampled to 4km and will be directly integrated into the 4km ARD. 
2. __Resampling TWC Hi-Res Precipitation (8km) Data to 4km:__ Resampling TWC High-Res 8km Precipitation Data to 4km. This TWC dataset starts from 2015-12-01. We will replace modeled precipitation dataset with these high resolution dataset starting from 2015-12-01.
3. __Replacing some modeled datasets with TWC datasets from 2015:__ We have TWC high resolution data records starting from 2015-06-30 (for precipitation from 2015-12-01). We have modeled high resolution data from 2002 to 2023 using ML/DL models. Despite that, from 2015 we will use TWC high resolution 4km data. Therefore, we will replace modeled data with TWC datasets from 2015 before compling all datasets into the 4km ARD.
4. __Compiling weather datasets to multiple `preARD` dataframes:__ Weather datasets have extended daily records, therefore, compiling weather datasets into multiple dataframe (referring to as `preARD`). These `preARD` dataframes will be combined into a single dataframe and averaged to form the 4km weather ARD.
5. __Generating weather ARD 4km:__ Compiling the `preARD` dataframes into a single dataframe. This single dataframe is our `4km resolution ARD`. In this final ARD, the datasets are average for each date `(month_day)` for all the years (2002-2023).

----------------

In [1]:
from ipynb.fs.full.ARD_utils import *

# V1 (with modeled precipitation)

### 1. Copying `7-9` datasets to `'../datasets/downscaled_data'` directory

In [7]:
# copying min relative humidity dataset
min_rel_humidity_dir = '../../datasets/weather_raster_data/cities_California/era5_data/resampled_4km_rasters_bilinear/min_Rhumid'
min_rel_humidity_copy_dir =  '../../datasets/downscaled_data/4km/weather_data/resampled_4km/min_Rhumid'
copy_file(input_dir_file=min_rel_humidity_dir, copy_dir=min_rel_humidity_copy_dir, search_by='*.tif')


# copying max relative humidity dataset
max_rel_humidity_dir = '../../datasets/weather_raster_data/cities_California/era5_data/resampled_4km_rasters_bilinear/max_Rhumid'
max_rel_humidity_copy_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/max_Rhumid'
copy_file(input_dir_file=max_rel_humidity_dir, copy_dir=max_rel_humidity_copy_dir, search_by='*.tif')


# copying total net solar radiation dataset
tot_net_SR_dir = '../../datasets/weather_raster_data/cities_California/era5_data/resampled_4km_rasters_bilinear/TotNet_SR'
tot_net_SR_copy_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/TotNet_SR'
copy_file(input_dir_file=tot_net_SR_dir, copy_dir=tot_net_SR_copy_dir, search_by='*.tif')

'../../datasets/downscaled_data/4km/weather_data/resampled_4km/TotNet_SR/TotNet_SR_20050707.tif'

### 2. Resampling TWC Hi-Res Precipitation (8km) Data to 4km 
Resampling TWC High-Res 8km Precipitation Data to 4km. This TWC dataset starts from 2015-12-01. We will replace modeled precipitation dataset with these high resolution dataset starting from 2015-12-01.

In [8]:
total_precip_8km_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/precip_data_8km/total_precip'
total_precip_4km_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/resampled_precip_data_4km/total_precip'

ref_raster = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/max_temp/max_temp_20020101.tif'

total_precip_8km_datasets = glob(os.path.join(total_precip_8km_dir_twc, '*.tif'))

# Resampling TWC Hi-Res data from 8km to 4km
for precip_data in total_precip_8km_datasets:
    raster_name = os.path.basename(precip_data)
    resample_raster_based_on_ref_raster(input_raster=precip_data, ref_raster=ref_raster, 
                                        output_dir=total_precip_4km_dir_twc, 
                                        raster_name=raster_name, resampling_alg=Resampling.bilinear,
                                        paste_value_on_ref_raster=False)

### 3. Replacing some modeled datasets with TWC datasets from 2015
We have TWC high resolution data records starting from 2015-06-30 (for precipitation from 2015-12-01). We have modeled high resolution data from 2002 to 2023. Despite that, from 2015 we will use TWC high resolution 4km data. Therefore, we will replace modeled data with TWC datasets from 2015 before compiling all datasets into the 4km ARD.

In [9]:
# TWC weather datasets
max_temp_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/weather_data_4km/max_temp'
min_temp_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/weather_data_4km/min_temp'
avg_rel_humid_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/weather_data_4km/avg_Rhumid'
avg_wind_speed_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/weather_data_4km/avg_wind_speed'
precipitation_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/resampled_precip_data_4km/total_precip'


# model interpolated weather datasets 
max_temp_dir_modeled = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/max_temp'
min_temp_dir_modeled = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/min_temp'
avg_rel_humid_dir_modeled = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_Rhumid'
avg_wind_speed_dir_modeled = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_wind_speed'
precipitation_dir_modeled = '../../datasets/downscaled_data/4km/weather_data/modeled_precipitation_resampled_4km/total_precip'  # ML/DL model generated total_precip 



copy_file(input_dir_file=max_temp_dir_twc, copy_dir=max_temp_dir_modeled, search_by='*.tif')
copy_file(input_dir_file=min_temp_dir_twc, copy_dir=min_temp_dir_modeled, search_by='*.tif')
copy_file(input_dir_file=avg_rel_humid_dir_twc, copy_dir=avg_rel_humid_dir_modeled, search_by='*.tif')
copy_file(input_dir_file=avg_wind_speed_dir_twc, copy_dir=avg_wind_speed_dir_modeled, search_by='*.tif')
copy_file(input_dir_file=precipitation_dir_twc, copy_dir=precipitation_dir_modeled, search_by='*.tif')

'../../datasets/downscaled_data/4km/weather_data/modeled_precipitation_resampled_4km/total_precip/total_precip_20181212.tif'

### 4. Compiling weather datasets to multiple `preARD` dataframes

Weather datasets have extended daily records, therefore, compiling weather datasets into multiple dataframe (referring to as `preARD`). These `preARD` dataframes will be combined into a single dataframe and averaged to form the 4km weather ARD.

In [2]:
# model interpolated weather datasets (have TWC datasets from 2015)
max_temp_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/max_temp'
min_temp_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/min_temp'
avg_temp_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_temp'
avg_rel_humid_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_Rhumid'
avg_wind_speed_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_wind_speed'
precipitation_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_precipitation_resampled_4km/total_precip'  # ML/DL model generated total_precip + TWC High-Res resampled 4km precip from 2015


# era5 resampled weather datasets
min_rel_humidity_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/min_Rhumid'
max_rel_humidity_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/max_Rhumid'
total_net_SR_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/TotNet_SR'


all_data_directories_v1 = [max_temp_dir, min_temp_dir, avg_temp_dir, avg_rel_humid_dir, avg_wind_speed_dir,
                        precipitation_dir, min_rel_humidity_dir, max_rel_humidity_dir, total_net_SR_dir]

In [3]:
all_data_directories_v1

['../../datasets/downscaled_data/4km/weather_data/modeled_4km/max_temp',
 '../../datasets/downscaled_data/4km/weather_data/modeled_4km/min_temp',
 '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_temp',
 '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_Rhumid',
 '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_wind_speed',
 '../../datasets/downscaled_data/4km/weather_data/modeled_precipitation_resampled_4km/total_precip',
 '../../datasets/downscaled_data/4km/weather_data/resampled_4km/min_Rhumid',
 '../../datasets/downscaled_data/4km/weather_data/resampled_4km/max_Rhumid',
 '../../datasets/downscaled_data/4km/weather_data/resampled_4km/TotNet_SR']

In [4]:
create_weather_4km_preARD_dataframes(data_directories_list=all_data_directories_v1, 
                                     output_folder='../../datasets/ARD/4km', savename='weather_4km_preARD_V1', 
                                     dataset_in_each_chunk=3)

compiling data for max_temp...
compiling data for min_temp...
compiling data for avg_Rhumid...
compiling data for avg_wind_speed...
compiling data for total_precip...
compiling data for avg_temp...
compiling data for min_Rhumid...
compiling data for max_Rhumid...
compiling data for TotNet_SR...


### 5. Generating weather ARD 4km
Compiling the preARD dataframes into a single dataframe. This single dataframe is our `4km resolution ARD V1`.

In [5]:
weather_ard_df_v1 = compile_preARD_multiDF_to_ARD(parquet_folder='../../datasets/ARD/4km/weather_4km_preARD_V1', 
                              output_folder='../../datasets/ARD/4km', savename='weather_4km_ARD_V1.parquet')

In [6]:
weather_ard_df_v1

,month_day,lat,lon,max_temp,min_temp,avg_Rhumid,avg_wind_speed,total_precip,avg_temp,min_Rhumid,max_Rhumid,TotNet_SR
0,01_01,35.813145,-119.858718,13.064774,2.556032,76.720100,6.198810,1.368499,8.057868,43.226334,86.756714,7.763757
1,01_01,35.813145,-119.822718,13.107390,2.549996,77.720039,5.814856,1.317763,8.107609,43.714951,86.954338,7.735141
2,01_01,35.813145,-119.786718,13.126426,2.521558,78.263412,5.647069,1.269205,8.155430,43.871330,87.074242,7.708403
3,01_01,35.813145,-119.750718,13.140491,2.656670,78.713570,5.349788,1.226165,8.125828,43.934681,87.172386,7.682190
4,01_01,35.813145,-119.714718,13.156439,2.594357,79.098198,5.344284,1.190915,8.106210,43.998032,87.270538,7.655977
...,...,...,...,...,...,...,...,...,...,...,...,...
556681,12_31,39.377145,-122.054718,13.111296,3.358657,71.012802,12.695121,3.108317,8.476252,38.840889,82.934311,7.148525
556682,12_31,39.377145,-122.018718,13.013983,3.376590,71.864647,12.712395,3.204798,8.512499,39.712254,83.251572,7.131769
556683,12_31,39.377145,-121.982718,12.994287,3.301607,72.312447,12.550989,3.305635,8.422517,40.583618,83.568840,7.115013
556684,12_31,39.377145,-121.946718,12.966456,3.310997,72.618156,12.385925,3.326128,8.344420,41.454987,83.886108,7.098257


In [25]:
## number of records in a particulat month_day
single_day_df = weather_ard_df_v1[weather_ard_df_v1['month_day']=='01_02']
len(single_day_df)

1521

`For a single mm_dd, there are 1521 rows of record each having 4km pixel size.`

--------------------------

------------------------

# V2 (with resampled precipitation)

As the precipitation model didn't perform well, we will use resampled (directly from 28km ERA5 data to 4km) precipitation data in the ARD and observe environmental clustering model performance.

### 1. Copying `6-9` datasets to `'../datasets/downscaled_data'` directory

In [8]:
# copying min relative humidity dataset
min_rel_humidity_dir = '../../datasets/weather_raster_data/cities_California/era5_data/resampled_4km_rasters_bilinear/min_Rhumid'
min_rel_humidity_copy_dir =  '../../datasets/downscaled_data/4km/weather_data/resampled_4km/min_Rhumid'
copy_file(input_dir_file=min_rel_humidity_dir, copy_dir=min_rel_humidity_copy_dir, search_by='*.tif')


# copying max relative humidity dataset
max_rel_humidity_dir = '../../datasets/weather_raster_data/cities_California/era5_data/resampled_4km_rasters_bilinear/max_Rhumid'
max_rel_humidity_copy_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/max_Rhumid'
copy_file(input_dir_file=max_rel_humidity_dir, copy_dir=max_rel_humidity_copy_dir, search_by='*.tif')


# copying total net solar radiation dataset
tot_net_SR_dir = '../../datasets/weather_raster_data/cities_California/era5_data/resampled_4km_rasters_bilinear/TotNet_SR'
tot_net_SR_copy_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/TotNet_SR'
copy_file(input_dir_file=tot_net_SR_dir, copy_dir=tot_net_SR_copy_dir, search_by='*.tif')


# copying total precipitataion (resampled from ERA5) dataset
total_precip_dir = '../../datasets/weather_raster_data/cities_California/era5_data/resampled_4km_rasters_bilinear/total_precip'
total_precip_copy_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/total_precip'
copy_file(input_dir_file=total_precip_dir, copy_dir=total_precip_copy_dir, search_by='*.tif')

'../../datasets/downscaled_data/4km/weather_data/resampled_4km/total_precip/total_precip_20181212.tif'

### 2. Resampling TWC Hi-Res Precipitation (8km) Data to 4km 
Resampling TWC High-Res 8km Precipitation Data to 4km. This TWC dataset starts from 2015-12-01. We will replace modeled precipitation dataset with these high resolution dataset starting from 2015-12-01.

This step was already performed for V1 data. Commenting it out

In [ ]:
# total_precip_8km_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/precip_data_8km/total_precip'
# total_precip_4km_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/resampled_precip_data_4km/total_precip'

# ref_raster = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/max_temp/max_temp_20020101.tif'

# total_precip_8km_datasets = glob(os.path.join(total_precip_8km_dir_twc, '*.tif'))

# # Resampling TWC Hi-Res data from 8km to 4km
# for precip_data in total_precip_8km_datasets:
#     raster_name = os.path.basename(precip_data)
#     resample_raster_based_on_ref_raster(input_raster=precip_data, ref_raster=ref_raster, 
#                                         output_dir=total_precip_4km_dir_twc, 
#                                         raster_name=raster_name, resampling_alg=Resampling.bilinear,
#                                         paste_value_on_ref_raster=False)

### 3. Replacing some modeled datasets with TWC datasets from 2015
We have TWC high resolution data records starting from 2015-06-30 (for precipitation from 2015-12-01). We have modeled high resolution data from 2002 to 2023. Despite that, from 2015 we will use TWC high resolution 4km data. Therefore, we will replace modeled data with TWC datasets from 2015 before compiling all datasets into the 4km ARD.

We already copied `max_temp`, `min_temp`, `avg_rel_humidity`, and `avg_wind_speed` TWC data to modeled data folder in V1 ARD procesing. Commenting Out those steps in the following block. Only copying resampled 4km TWC High-Res Precipitation data to resampled ERA5 Precipitation folder.

In [7]:
# TWC weather datasets
# max_temp_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/weather_data_4km/max_temp'
# min_temp_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/weather_data_4km/min_temp'
# avg_rel_humid_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/weather_data_4km/avg_Rhumid'
# avg_wind_speed_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/weather_data_4km/avg_wind_speed'
precipitation_dir_twc = '../../datasets/weather_raster_data/cities_California/twc_data/resampled_precip_data_4km/total_precip'


# model interpolated weather datasets
# max_temp_dir_modeled = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/max_temp'
# min_temp_dir_modeled = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/min_temp'
# avg_rel_humid_dir_modeled = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_Rhumid'
# avg_wind_speed_dir_modeled = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_wind_speed'

## ERA5 resampled precipitation folder
precipitation_dir_era5_resampled = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/total_precip'  # era5 resampled total_precip (2002-2015) + TWC resampled precip after 2015 



# copy_file(input_dir_file=max_temp_dir_twc, copy_dir=max_temp_dir_modeled, search_by='*.tif')
# copy_file(input_dir_file=min_temp_dir_twc, copy_dir=min_temp_dir_modeled, search_by='*.tif')
# copy_file(input_dir_file=avg_rel_humid_dir_twc, copy_dir=avg_rel_humid_dir_modeled, search_by='*.tif')
# copy_file(input_dir_file=avg_wind_speed_dir_twc, copy_dir=avg_wind_speed_dir_modeled, search_by='*.tif')
copy_file(input_dir_file=precipitation_dir_twc, copy_dir=precipitation_dir_era5_resampled, search_by='*.tif')

'../../datasets/downscaled_data/4km/weather_data/resampled_4km/total_precip/total_precip_20181212.tif'

### 4. Compiling weather datasets to multiple `preARD` dataframes

Weather datasets have extended daily records, therefore, compiling weather datasets into multiple dataframe (referring to as `preARD`. These `preARD` dataframes will be combined into a single dataframe and averaged to form the 4km weather ARD.

In [8]:
# model interpolated weather datasets (have TWC datasets from 2015)
max_temp_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/max_temp'
min_temp_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/min_temp'
avg_temp_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_temp'
avg_rel_humid_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_Rhumid'
avg_wind_speed_dir = '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_wind_speed'


# era5 resampled weather datasets
min_rel_humidity_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/min_Rhumid'
max_rel_humidity_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/max_Rhumid'
total_net_SR_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/TotNet_SR'
precipitation_dir = '../../datasets/downscaled_data/4km/weather_data/resampled_4km/total_precip'  ###### resampled total_precip (not ML/DL model interpolated) + TWC High-Res resampled 4km precip from 2015


all_data_directories_v2 = [max_temp_dir, min_temp_dir, avg_temp_dir, avg_rel_humid_dir, avg_wind_speed_dir,
                        min_rel_humidity_dir, max_rel_humidity_dir, total_net_SR_dir, precipitation_dir]

In [9]:
all_data_directories_v2

['../../datasets/downscaled_data/4km/weather_data/modeled_4km/max_temp',
 '../../datasets/downscaled_data/4km/weather_data/modeled_4km/min_temp',
 '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_temp',
 '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_Rhumid',
 '../../datasets/downscaled_data/4km/weather_data/modeled_4km/avg_wind_speed',
 '../../datasets/downscaled_data/4km/weather_data/resampled_4km/min_Rhumid',
 '../../datasets/downscaled_data/4km/weather_data/resampled_4km/max_Rhumid',
 '../../datasets/downscaled_data/4km/weather_data/resampled_4km/TotNet_SR',
 '../../datasets/downscaled_data/4km/weather_data/resampled_4km/total_precip']

In [10]:
create_weather_4km_preARD_dataframes(data_directories_list=all_data_directories_v2, 
                                     output_folder='../../datasets/ARD/4km', savename='weather_4km_preARD_V2', 
                                     dataset_in_each_chunk=3)

compiling data for max_temp...
compiling data for min_temp...
compiling data for avg_Rhumid...
compiling data for avg_wind_speed...
compiling data for total_precip...
compiling data for avg_temp...
compiling data for min_Rhumid...
compiling data for max_Rhumid...
compiling data for TotNet_SR...


### 5. Generating weather ARD 4km
Compiling the preARD dataframes into a single dataframe. This single dataframe is our `4km resolution ARD V2`.

In [11]:
weather_ard_df_v2 = compile_preARD_multiDF_to_ARD(parquet_folder='../../datasets/ARD/4km/weather_4km_preARD_V2', 
                              output_folder='../../datasets/ARD/4km', savename='weather_4km_ARD_V2.parquet')

In [12]:
weather_ard_df_v2

,month_day,lat,lon,max_temp,min_temp,avg_Rhumid,avg_wind_speed,total_precip,avg_temp,min_Rhumid,max_Rhumid,TotNet_SR
0,01_01,35.813145,-119.858718,13.064774,2.556032,76.720100,6.198810,1.415220,8.057868,43.226334,86.756714,7.763757
1,01_01,35.813145,-119.822718,13.107390,2.549996,77.720039,5.814856,1.341458,8.107609,43.714951,86.954338,7.735141
2,01_01,35.813145,-119.786718,13.126426,2.521558,78.263412,5.647069,1.292602,8.155430,43.871330,87.074242,7.708403
3,01_01,35.813145,-119.750718,13.140491,2.656670,78.713570,5.349788,1.253974,8.125828,43.934681,87.172386,7.682190
4,01_01,35.813145,-119.714718,13.156439,2.594357,79.098198,5.344284,1.219387,8.106210,43.998032,87.270538,7.655977
...,...,...,...,...,...,...,...,...,...,...,...,...
563635,12_31,39.377145,-121.910718,12.920015,3.277704,73.235123,12.123603,3.627121,8.315034,42.326351,84.203377,7.081501
563636,12_31,39.413145,-122.126718,12.778427,3.334260,70.784737,11.975956,2.754346,8.021772,40.393192,82.094841,7.157553
563637,12_31,39.413145,-122.090718,12.671306,3.222568,71.297905,12.386173,2.702650,7.976338,40.296097,82.996353,7.175453
563638,12_31,39.413145,-122.054718,12.740860,3.307436,71.610085,12.684472,2.705669,8.043331,40.397404,83.751907,7.184686
